In [1]:
# Import Python package
import BinhoSupernova
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.system.definitions import GetUsbStringSubCommand

## Getting started

#### 1. List all the Supernova devices connected to the PC host

The ``BinhoSupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [2]:
BinhoSupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#7&8ada5c5&1&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '12B24708C123895DB326EDA7E1FBA6D',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [3]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [4]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:

    if supernova_message != None:

        # Print mesasage
        print(">> New message from SUPERNOVA:")
        print(supernova_message)

    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

## System API

``Supernova.resetDevice(id)``

This method resets the Supernova. As a result an `UsbDisconnectionError Exception` will be raised when the devices is reset. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.


In [ ]:
supernova.resetDevice(getId())

``Supernova.enterBootMode(id)``

This method makes the Supernova jump to the USB bootloader. As a result an `UsbDisconnectionError Exception` will be raised when the devices is reset and then a new USB mass storage device should be enumerated by the OS whose name should be **SUPERNOVA**. 

- ``id: int`` : A 2-byte integer that represents the transfer ID.


In [ ]:
supernova.enterBootMode(getId())

``Supernova.getUsbString(id, subCommand)``

The ``GET USB STRING`` command allows you to retrieve various string values related to the product, such as the manufacturer, product name, serial number, firmware version, and hardware version. This command serves as a basic test to verify the communication and functionality with the Supernova device. By successfully receiving the expected string values, you can ensure that the communication between the PC host and the Supernova device is functioning as expected.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``subCommand: GetUsbStringSubCommand``: The subcommand parameter indicates which string must be returned by the device.

In [8]:
# GET USB STRING - Get Manufacturer
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.MANUFACTURER)

# GET USB STRING  - Get Product Name
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.PRODUCT_NAME)

# GET USB STRING  - Get Serial Number
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.SERIAL_NUMBER)

# GET USB STRING  - Firmware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

# GET USB STRING  - Hardware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.HW_VERSION)

>> New message from SUPERNOVA:
{'id': 1, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 12, 'payload': 'MN-Binho LLC'}
>> New message from SUPERNOVA:
{'id': 2, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 18, 'payload': 'PR-Binho Supernova'}
>> New message from SUPERNOVA:
{'id': 3, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 35, 'payload': 'SN-12B24708C123895DB326EDA7E1FBA6D6'}
>> New message from SUPERNOVA:
{'id': 4, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 20, 'payload': 'FW-3.2.0-275-1e81ddc'}
>> New message from SUPERNOVA:
{'id': 5, 'command': 'SYS GET USB STRING', 'result': 'SUCCESS', 'payload_length': 4, 'payload': 'HW-C'}


``Supernova.setI3cVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the I3C bus in mV, ranging from 800 mV up to 3300 mV. It also allows setting the value 0 mV to power off the output voltage.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [800, 3300] mV and allows the value 0 mV to power off the output voltage.

In [9]:
supernova.setI3cVoltage(getId(), 3300)

{'module': 1, 'opcode': 0, 'message': 'SET I3C VOLTAGE request success'}

>> New message from SUPERNOVA:
{'id': 6, 'command': 'SYS SET I3C VOLTAGE', 'result': 'SUCCESS'}


``Supernova.setI2cSpiUartGpioVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the bus in mV, ranging from 1200 mV up to 3300 mV. It also allows setting the value 0 mV to power off the output voltage.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [1200, 3300] mV and allows the value 0 mV to power off the output voltage.

In [10]:
supernova.setI2cSpiUartGpioVoltage(getId(), 3300)

{'module': 1,
 'opcode': 0,
 'message': 'SET I2C SPI UART GPIO VOLTAGE request success'}

>> New message from SUPERNOVA:
{'id': 7, 'command': 'SYS SET I2C SPI UART GPIO VOLTAGE', 'result': 'SUCCESS'}
>> New message from SUPERNOVA:
{'id': 0, 'command': 'SYS I3C CONNECTOR EVENT NOTIFICATION', 'result': 'SUCCESS', 'event': 'I3C_CONNECTOR_PLUGGED', 'port': 'I3C_LOW_VOLTAGE_PORT', 'connector_type': 'I3C_HARNESS'}


``Supernova.getI3cConnectorStatus(id)``

This method returns the status of both I3C ports, including whether or not a connector is plugged, and if so, which connector type and in which port.

- ``id: int`` : A 2-byte integer that represents the transfer ID.

In [11]:
supernova.getI3cConnectorsStatus(getId())

{'module': 1,
 'opcode': 0,
 'message': 'GET I3C CONNECTORS STATUS request success'}

>> New message from SUPERNOVA:
{'id': 8, 'command': 'SYS GET I3C CONNECTOR STATUS', 'result': 'SUCCESS', 'i3c_low_voltage_port': {'state': 'I3C_CONNECTOR_PLUGGED', 'connector_type': 'I3C_HARNESS'}, 'i3c_high_voltage_port': {'state': 'I3C_CONNECTOR_UNPLUGGED', 'connector_type': 'NO_CONNECTOR'}}


``Supernova.getAnalogMeasurements(id)``

This method returns the voltage of all the internal and external power domains, in mV.

- ``id: int`` : A 2-byte integer that represents the transfer ID.

In [12]:
supernova.getAnalogMeasurements(getId())

{'module': 1,
 'opcode': 0,
 'message': 'GET ANALOG MEASUREMENTS request success'}

>> New message from SUPERNOVA:
{'id': 9, 'command': 'SYS GET ANALOG MEASUREMENTS', 'result': 'SUCCESS', 'i2c_spi_uart_gpio_vtarg': {'internal_mV': 3299, 'external_mV': 3299}, 'i3c_low_voltage_vtarg': {'internal_mV': 1310, 'external_mV': 2}, 'i3c_high_voltage_vtarg': {'internal_mV': 3299, 'external_mV': 3299}}


``Supernova.useExternalI3cVoltage(id)``

This method indicates that an external voltage source is used to supply the I3C bus and the Supernova must set its internal voltage to meet the external voltage. This methods returns the voltage measured in the VTARG pins.

- ``id: int`` : A 2-byte integer that represents the transfer ID.

In [13]:
supernova.useExternalI3cVoltage(getId())

{'module': 1,
 'opcode': 0,
 'message': 'USE EXT SRC I3C VOLTAGE request success'}

>> New message from SUPERNOVA:
{'id': 10, 'command': 'SYS USE EXTERNAL I3C VOLTAGE', 'result': 'I3C_PORTS_NOT_POWERED', 'i3c_low_voltage_mV': 0, 'i3c_high_voltage_mV': 1}


``Supernova.useExternalI2cSpiUartGpioVoltage(id)``

This method indicates that an external voltage source is used to supply the I2C, SPI and UART buses as well as GPIOs interface, and the Supernova must set its internal voltage to meet the external voltage. This methods returns the voltage measured in the VTARG pin.

- ``id: int`` : A 2-byte integer that represents the transfer ID.

In [14]:
supernova.useExternalI2cSpiUartGpioVoltage(getId())

{'module': 1,
 'opcode': 0,
 'message': 'USE EXT SRC I2C-SPI-UART-GPIO VOLTAGE request success'}

>> New message from SUPERNOVA:
{'id': 11, 'command': 'SYS USE EXTERNAL I2C SPI UART GPIO VOLTAGE', 'result': 'VOLTAGE_OUT_OF_RANGE', 'voltage_mV': 2}


## Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [15]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}